Import Core Modules

** Note: This module computes the absolute value of factorized waveform modes in a manner optimized for flux calculations for the initial conditions and the evolution of the equations of motion. A different module will encapsulate phase terms that factor into the full complex factorized waveform.

In [1]:
import numpy as np
import os.path
from scipy.optimize import root
from scipy.interpolate import interp1d, interp2d
from numpy.linalg import norm
#import SEOBNR.NQC_corrections as nqc
#import SEOBNR.nqc_interp as nqi

# For testing, remove numpy and sympy expression files
# For now, do NOT regenerate CSE expressions
import shutil, os
import sys#TylerK: Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
outdir = os.path.join("Flux/")
codesdir = "Flux"
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface


## The Factorized Waveform

The Factorized Waveform in the SEOBNR formalism is given by:

$$
h^{\rm F}_{lm} = h^{(N,\epsilon)}_{lm} \hat{S}^{(\epsilon)}_{\rm eff} T_{lm} (\rho_{lm})^l
$$

Where, $h^{(N,\epsilon)}_{lm}$ are the Newtonian Modes, $\hat{S}^{(\epsilon)}_{\rm eff}$ is an effective source term, $T_{lm}$ encodes the logarithmic tail terms, and $\rho_{lm}$ encodes the PN corrections to the Newtonian waveform. Finally, the label $\epsilon$ is the parity of the waveform.

In [2]:
%%writefile $codesdir/Factorized_Waveform_For_Flux-on-top.txt

hF_lm = hNe_lm*Se_eff*T_lm*rholmpowl

Writing Flux/Factorized_Waveform_For_Flux-on-top.txt


### The Newtonian Modes

$$
h^{(N,\epsilon)}_{lm} = \eta n^{(\epsilon)}_{lm} c_{l+\epsilon} V^l_{\Phi} Y^{l - \epsilon, -m}\left(\frac{\pi}{2},\Phi\right)
$$

Where, $\eta$ is the symmetric mass ratio, $Y$ represents the scalar spherical harmonics, $n^{(\epsilon)}_{lm}$ and $c_{l+\epsilon}$ are numerical coefficients, and $V^l_{\Phi}$ is the velocity contribution.

In [3]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

hNe_lm = eta*ne_lm*c_lpluse*Vl_Phi*Ylminuse_minusm

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Newtonian Prefix $n^{(\epsilon)}_{lm}$

We define the Newtonian prefix in a different function as part of an auxiliary python module.

In [4]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

ne_lm = aux.Newtonian_n(m1,m2,l,m)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Newtonian Prefix $c_{l+\epsilon}$

We define the Newtonian prefix in a different function as part of an auxiliary python module.

In [5]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

c_lpluse = aux.Newtonian_c(m1,m2,l,m)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Spherical Harmonic $Y$

The scalar spherical harmonics functions evaluated at the equator are defined in a different function as part of an auxiliary python module.

In [6]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Ylminuse_minusm = aux.AbsSphericalHarmonicAtPiOver2(l-epsilon,-m)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Velocity Contribution $V^l_{\Phi}$

The velocity contribution to the Newtonian Mode is given by:
$$
V^l_{\Phi} = v^{l+\epsilon}_{\Phi}
$$

Where, $v_{\Phi}$ is the azimuthal velocity and $\epsilon$ is the parity of the waveform mode.

In [7]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Vl_Phi = np.power(vPhi,l+epsilon)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Azimuthal Velocity $v_\Phi$

The Azimuthal velocity $v_\Phi$ is given by:

$$
v_\Phi = \Omega v^{(\mathrm{NK})}_\Phi
$$

Where $\Omega$ is the orbital frequency, and $v^{(\mathrm{NK})}_\Phi$ is the non-Keplerian correction to the velocity which is calculated in a different function in an auxiliary python module.

In [8]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

vPhi = Omega*aux.vPhiNonKeplerian(m1,m2,EMgamma,tortoise,q,p,S1,S2)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


## Logarithmic Tail Term $T_{lm}$

The logarithmic tail terms are given by their resummed form in DNB(1212.4357):

$$
T_{lm} = \frac{1}{l!}\sqrt{\frac{4\pi\hat{\hat{k}}}{1 - e^{-4\pi\hat{\hat{k}}}}}\sqrt{\prod_{s=1}^{l}\left( s^2 + \left(2\hat{\hat{k}}\right)^2 \right)}
$$

Where, $\hat{\hat{k}}$ is the Hamiltonian contribution.

In [9]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

T_lm = lfactorialinv*Tlmprefac*np.sqrt(Tlmprodfac)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


$T_{lm}$ Product Factor

The product factor entering into the definition of $T_{lm}$ are defined in a different function as part of an auxiliary python module.

In [10]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Tlmprodfac = aux.Tlmprodfac(l,hathatk)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


$T_{lm}$ Prefactor

We define the prefactor as:

$$
T_{lm}^{(\mathrm{pre})} = \sqrt{\frac{4\pi\hat{\hat{k}}}{1 - e^{-4\pi\hat{\hat{k}}}}}
$$

In [11]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Tlmprefac = np.sqrt( pihathatk4 / ( 1 - np.exp( -pihathatk4 ) ) )

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Coefficient pihathatk4

We define here $4\pi$ times the Hamiltonian factor $\hat{\hat{k}}$

In [12]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

pihathatk4 = 4*np.pi*hathatk

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The Hamiltonian Factor hathatk

The Hamiltonian factor is given as:

$$
\hat{\hat{k}} = m H_\mathrm{real} \Omega
$$

Where, $H_\mathrm{real}$ is the Effective-One-Body Hamiltonian, and $\Omega$ is the orbital frequency.

In [13]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

hathatk = m*Hreal*Omega

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The frequency $\Omega$

The orbital frequency $\Omega$ is defined in terms of the velocity $v$ as:

$$
\Omega = v^3
$$

In [14]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Omega = np.power(v,3)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The term $\frac{1}{l!}$

In [15]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

lfactorialinv = 1/np.math.factorial(l)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


## The Source Term $\hat{S}^{(\epsilon)}_{\mathrm{eff}}$

The effective source term is also computed in a different function as part of an auxiliary python module.

In [16]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

Se_eff = aux.Se_eff(l,m,Hreal,v,q,p,eta)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


## The PN Corrections $(\rho_{lm})^l$

This term is simply the PN Corrections $\rho_{lm}$ raised to the power of $l$

In [17]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

rholmpowl = aux.rholmpowl(m1,m2,l,m,chiA,chiS,v,EMgamma)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The parity of the Waveform mode $\epsilon$

We define $\epsilon$ as the parity of the waveform

$$
\epsilon = (l+m)\mathbf{mod}2
$$

In [18]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

epsilon = (l+m)%2

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


The symmetric mass ratio $\eta$

The symmetric mass ratio $\eta$ is defined as follows
$$
\eta = \frac{m_1m_2}{(m_1 + m_2)^2}
$$

In [19]:
%%writefile -a $codesdir/Factorized_Waveform_For_Flux-on-top.txt

eta = m1*m2/(m1+m2)/(m1+m2)

Appending to Flux/Factorized_Waveform_For_Flux-on-top.txt


Compile this into a python module

In [20]:
# Now reverse the expressions and write them in a function

with open(os.path.join(outdir,"factorized_waveform_for_flux.py"), "w") as output:
    output.write("import numpy as np\nimport auxiliaryfunctions as aux\ndef compute_hFlm_for_flux(m1, m2, EMgamma, tortoise, q, p, S1, S2, l, m, Hreal, v,chiA,chiS):\n")
    for line in reversed(list(open("Flux/Factorized_Waveform_For_Flux-on-top.txt"))):
        output.write("    %s\n" % line.rstrip())
    output.write("    return hF_lm")
